In [ ]:
import torch
from torch import nn
import numpy as np
from functools import partial
import matplotlib.pyplot as plt


### **Константы флюида:** ###


In [ ]:
p_0 = 117.3  # [p] = бар
T = 82.2  # [T] = oC
d = 0.1524


In [ ]:
gamma_g = 0.75
p_sep = 1.013  # [p_sep] = бар
T_sep = 15.6  # [T_sep] = oC
gamma_api = 33
R_p = 178  # [R_p] = м^3 газа / м^3 нефти
gamma_gd = 0.88
Z = 0.853


### **Расчет параметров массобмена и физических свойств флюида в рамках модели нелетучей нефти:** ###


#### 1. _Отношение растворенного газа к нефти (по методу Васкеса и Беггза):_ ####


In [ ]:
gamma_g100 = gamma_g * (1.0 + 5.912 * (10 ** (-5)) *
                        gamma_api * (1.8 * T_sep + 32) * np.log10(p_sep / 7.9))


#### 2. _Корреляция Васкеса и Беггза для определения растворимости:_ ####


In [ ]:
if gamma_api > 30:
    C1_Rs = 0.0178
    C2_Rs = 1.1870
    C3_Rs = 23.931
else:
    C1_Rs = 0.0362
    C2_Rs = 1.0937
    C3_Rs = 25.7245

def R_s(p):
    return 0.178 * C1_Rs * gamma_g100 * (np.sign(14.51) * (np.abs(14.51)) ** (C2_Rs)) * (np.sign(p) * (np.abs(p)) ** (C2_Rs)) * np.exp(C3_Rs * gamma_api / (1.8 * T + 492))


#### 3. _Объемный коэффициент нефти (по методу Васкеса и Беггза):_ ####


In [ ]:
if gamma_api > 30:
    C1_Bo = 4.670 * (10 ** (-4))
    C2_Bo = 1.100 * (10 ** (-5))
    C3_Bo = 1.337 * (10 ** (-9))
else:
    C1_Bo = 4.677 * (10 ** (-4))
    C2_Bo = 1.751 * (10 ** (-5))
    C3_Bo = -1.811 * (10 ** (-8))


def B_0(p):
    return 1.0 + 5.618 * C1_Bo * R_s(p) + (1.8 * T - 28) * (gamma_api / gamma_g100) * (C2_Bo + 5.618 * C3_Bo * R_s(p))


#### 4. _Плотность нефти_ ####

In [ ]:
def gamma_gf(p): return (R_p * gamma_g - R_s(p) * gamma_gd) / (R_p - R_s(p))


gamma_0 = 141.5 / (gamma_api + 131.5)


def ro_o(p): return 16.02 * (62.4 * gamma_0 +
                             0.0764 * R_s(p) * gamma_gd) / B_0(p)


#### 5. _Псевдокритические давление и температура (по методу Стэндинга):_ ####


In [ ]:
T_pc = (168 + 325 * gamma_g - 12.5 * (gamma_g ** 2)) / 1.8
p_pc = 0.0689 * (677 + 15.0 * gamma_g - 37.5 * (gamma_g ** 2))


#### 6. _Коэффициент сжимаемости газа (по методу Стэндинга и Каца):_ ####


In [ ]:
T_pr = (T + 273.2) / T_pc  # [T] = K
def p_pr(p): return p / p_pc


#### 7. _Объемный коэффициент газа_ ####


In [ ]:
def B_g(p): return 3.511 * (10 ** (-3)) * Z * (T + 273.2) / p  # [T] = K


#### 8. _Плотность газа_ ####

In [ ]:
def ro_g(p): return 3.5 * (10 ** (-3)) * 0.75 * \
    (p * (10 ** 5)) / (Z * (T + 273.2))  # [p] = Па, [T] = K


### **Приведенные скорости: модель нелетучей нефти:** ###

In [ ]:
q_o_st = 1590
q_g_st =  283000

In [ ]:
A_p = np.pi / 4 * (d ** 2)


In [ ]:
def q_o(p): return q_o_st * B_0(p) / 86400
def q_g(p): return (q_g_st - q_o_st * R_s(p)) * B_g(p) / 86400


In [ ]:
def v_Sl(p): return q_o(p) / A_p
def v_Sg(p): return q_g(p) / A_p
def v_m(p): return v_Sl(p) + v_Sg(p)


### **Расчет вертикального градиента давления по методу Данса и Роса на основе данных многофазного потока**: ###

In [ ]:
mu_o = 0.97 * (10 ** (-3))
sigma_o = 8.41 * (10 ** (-3))
eps = 18.288 * (10 ** (-6)) 
g = 9.8

*Для пузырькового режима потока:*

In [ ]:
F1 = 1.2
F2 = 0.24
F3 = 1.3
F4 = 26.5
N_d = 143.8
N_gv = 11.54
N_Lv = 11.87
f1 = 0.0175
f2 = 1.0


In [ ]:
F3_1 = F3 - F4 / N_d
S = F1 + F2 * N_Lv + F3_1 * ((N_gv / (1 + N_Lv)) ** 2)

# def v_s(p): return S / ((ro_o(p) / (g * sigma_o)) ** (0.25))
def v_s(p): return S / (np.sign(ro_o(p) / (g * sigma_o)) * (np.abs((ro_o(p) / (g * sigma_o)))) ** (1/4))
def H_L(p): return (v_s(p) - v_m(p) +
                    np.sqrt((v_m(p) - v_s(p)) ** 2 + 4 * v_s(p) * v_Sl(p)))


def ro_s(p): return ro_o(p) * H_L(p) + ro_g(p) * (1 - H_L(p))


In [ ]:
def N_ReL(p): return ro_o(p) * v_Sl(p) * d / mu_o


# def f(p): return f1 * f2 / (1 + f1 / 4 * np.sqrt(v_Sg(p) / (50 * v_Sl(p))))
def f(p): return f1 * f2 / (1 + f1 / 4 * np.sign(v_Sg(p) / (50 * v_Sl(p)))
                            * (np.abs(v_Sg(p) / (50 * v_Sl(p)))) ** (1/2))


#### **Численное решение ОДУ:** ####

In [ ]:
from scipy.integrate import solve_ivp


def friction(z, p):
    return -(ro_s(p) + f(p) * ro_o(p) * v_Sl(p) * v_m(p) / (2 * d * g)) * 0.0000981

In [ ]:
N = 100
z_interval = [0, 10]
z = torch.linspace(z_interval[0], z_interval[1],
                   steps=N, requires_grad=True)
z = z.reshape(z.shape[0], 1)
sol = solve_ivp(friction, z_interval, y0=[p_0], t_eval=z.detach().numpy().reshape(-1), method='RK45')
sol.y = sol.y.reshape(-1)

In [ ]:
plt.plot(sol.t, sol.y)

### **PINN** ###

In [ ]:
class Net(nn.Module):
    """
    Класс для построения нейронной сети
    """

    def __init__(self, num_hidden, size_hidden, act=nn.Tanh()):
        """
        Архитектура нейронной сети

        :param num_hidden: количество скрытых слоев
        :param size_hidden: количество скрытых нейронов
        :param act: функция активации
        """
        super().__init__()

        self.layer_in = nn.Linear(1, size_hidden)  # 1 входное значение

        num_middle = num_hidden - 1
        self.middle_layers = nn.ModuleList(
            [nn.Linear(size_hidden, size_hidden) for _ in range(num_middle)])
        self.act = act  # функция активации

        self.layer_out = nn.Linear(size_hidden, 1)  # 1 выходное значение

    def forward(self, x):
        """
        Функция распространения

        :param x: входное значение x
        :return: выходное значение
        """
        out = self.act(self.layer_in(x))
        for layer in self.middle_layers:
            out = self.act(layer(out))
        return self.layer_out(out)


In [ ]:
def F(nn, x):
    """
    Значение приближенного решения NN

    :param nn: экземпляр нейронной сети (NN)
    :param x: значение  переменной x
    :return: значение приближенного решения
    """
    return nn(x)

In [ ]:
def df(nn, x=None, order=1):
    """
    Значение производной

    :param nn: экземпляр нейронной сети (NN)
    :param x: значение  переменной x
    :param order: порядок производной
    :return: значение прозводной
    """
    df_value = F(nn, x)
    for _ in range(order):
        df_value = torch.autograd.grad(
            df_value,
            x,
            grad_outputs=torch.ones_like(x),
            create_graph=True,
            retain_graph=True,
        )[0]
    return df_value

In [ ]:
def compute_loss(nn, x=None, verbose=False):
    """
    Функция потерь: L = L_de + L_bc

    :param nn: экземпляр нейронной сети (NN)
    :param x: значение  переменной x
    :param verbose: параметр
    :return: значение потерь
    """
    p_current = F(nn, x).detach().numpy()
    # Внутренние потери
    friction = ro_s(p_current) + f(p_current) * ro_o(p_current) * v_Sl(p_current) * v_m(p_current) / (2 * d * g)
    friction = torch.Tensor(friction)
    friction.requires_grad = True
    interior_loss = df(nn, x) + friction * 0.0000981
    # Потери на начальном условии
    boundary = torch.Tensor([0.])
    boundary.requires_grad = True
    boundary_loss = F(nn, boundary) - p_0

    final_loss = interior_loss.pow(2).mean() + boundary_loss.pow(2).mean() 
    return final_loss

In [ ]:
def train_model(nn, loss_function, learning_rate, max_epochs):
    """
    Тренировка нейронной сети

    :param nn: экземпляр нейронной сети (NN)
    :param loss_function: функция потерь
    :param learning_rate: скорость обучения
    :param max_epochs: число эпох
    :return: экземпляр сети, массив значений loss
    """
    loss_evolution = []
    # Оптимизация
    optimizer = torch.optim.Adam(nn.parameters(), lr=learning_rate)
    # Главный тренировочный цикл
    for epoch in range(max_epochs):
        loss = loss_function(nn)  # функция потерь
        optimizer.zero_grad()  # обнуление (перезапуск) градиентов
        loss.backward()  # обратное распространение ошибки
        optimizer.step()  # шаг оптимизатора

        if epoch % 1000 == 0:
            print(f"Epoch: {epoch} -> Loss: {float(loss)}")

        loss_evolution.append(loss.detach().numpy())
    return nn, np.array(loss_evolution)

In [ ]:
def plot_solution(net_solution, z_eval):
    fig, ax = plt.subplots(figsize=(10, 5), nrows=1, ncols=2)
    ax[0].plot(z_eval, net_solution, label="NN solution")
    ylim_min = min(net_solution) - 0.1
    ylim_max = max(net_solution) + 0.1
    ax[0].set_ylim([ylim_min, ylim_max])
    ax[0].set(title="Pressure", xlabel="Z", ylabel="p")
    ax[0].legend()

    ax[1].semilogy(loss_evolution)
    ax[1].set(title="Loss evolution", xlabel="Epoch", ylabel="Loss")


In [ ]:
def error(analytic_solution, net_solution):
    error = np.fabs(analytic_solution - net_solution)
    print(f"Error: {np.max(error)}")
    print("Outlet pressure:")
    print(
        f"**** NN: {net_solution[-1]} \n**** Analytic: {analytic_solution[-1]} \n**** Differnce: {np.fabs(net_solution[-1] - analytic_solution[-1])}")


In [ ]:
N = 100
z_interval = [0, 10]
num_hidden =5 # 2, 3, 5
size_hidden = 10 # 5, 10, 20, 40
lr = 0.05
epochs = 10000 # 5000, 10000, 20000


In [ ]:
z = torch.linspace(z_interval[0], z_interval[1],
                   steps=N, requires_grad=True)
z = z.reshape(z.shape[0], 1)

# torch.manual_seed(10)

# Инициализация нейронной сети
net = Net(num_hidden, size_hidden, act=nn.Tanh())

# net.apply(init_weights)

print("Архитектура сети: \n", net, end='\n\n')
# Тренировка нейронной сети
loss_function = partial(compute_loss, x=z, verbose=True)
net_trained, loss_evolution = train_model(
    net, loss_function=loss_function, learning_rate=lr, max_epochs=epochs)

# Решение, полученное тренировкой сети
z_eval = torch.linspace(z_interval[0], z_interval[1], steps=N).reshape(-1, 1)
net_solution = F(net_trained, z_eval)
print()
error(sol.y, net_solution.detach().numpy().reshape(-1))

In [ ]:
plot_solution(net_solution.detach().numpy(), z_eval)

### **Оценка решения:** ###

In [ ]:
error(net_solution.detach().numpy().reshape(-1), sol.y)